In [1]:
#  Add root folder to python path
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import numpy as np 
import pandas as pd
import scipy.stats as ss

from pof.data.asset_data import SimpleFleet

In [2]:
# Forecast years
START_YEAR = 2015
END_YEAR = 2024
CURRENT_YEAR = 2020

# Population Data
file_path = r"C:\Users\gtreseder\OneDrive - KPMG\Documents\3. Client\Essential Energy\Probability of Failure Model\inputs" + os.sep
FILE_NAME = r"population_summary.csv"

sfd = SimpleFleet(file_path + FILE_NAME)
sfd.load()
sfd.calc_forecast_age(START_YEAR, END_YEAR, CURRENT_YEAR)

,age,year,assets
0,1,2015,15092.0
1,2,2015,13549.0
2,3,2015,13101.0
3,4,2015,12181.0
4,5,2015,10345.0
...,...,...,...
985,95,2024,0.0
986,96,2024,5.0
987,97,2024,1.0
988,98,2024,15.0


## Check if it plays well with component

In [3]:
from pof import Component

In [4]:
t_end = 100
n_iterations = 100

comp = Component.demo()
comp.mc_timeline(t_end=t_end, n_iterations=n_iterations)

100%|██████████| 100/100 [00:12<00:00,  8.06it/s]


In [5]:
comp.expected_forecast_table(sfd.df_age)

,fm,ie (%),cf:ff (%),is,ff,cf,ff_pop_annual_avg,cf_pop_annual_avg,ff_pop,cf_pop,total
0,early_life,18,:,100,0,0,0,0,0,0,NaN
1,random,18,:,94,6,0,598.29,0,70495.7,0,NaN
2,slow_aging,95.4,66.67:33.33,85,10,5,440.07,349.13,117493,58746.4,NaN
3,fast_aging,95.4,86.67:13.33,70,26,4,1588.99,124.78,305482,46997.2,NaN
4,total,2.94878,82.35:17.65,49,42,9,NaN,NaN,986940,211487,1174929.0


In [31]:
cohort = sfd.df_age
""" Reports a summary for each of the failure modes and the expected outcomes over the MC simulation"""
# Get the key data from each of the failuremodes
ff_cf = {}

for fm in comp.fm.values():
    if fm.active:
        insp_effective = fm.inspection_effectiveness()
        _ff = fm.expected_ff()
        _cf = fm.expected_cf()

        ff_cf[fm.name] = {
            "fm": fm.name,
            "ie": insp_effective,
            "is": comp._sim_counter - len(_cf) - len(_ff),
            "ff": len(_ff),
            "cf": len(_cf),
        }

        if cohort is not None:
            age, count = np.unique(_cf, return_counts=True)
            ff_cf[fm.name]['cf_pop_annual_avg'] = cohort.reindex(age).mul(count, axis=0).sum()[0] / comp._sim_counter
            
            age, count = np.unique(_ff, return_counts=True)
            ff_cf[fm.name]['ff_pop_annual_avg'] = cohort.reindex(age).mul(count, axis=0).sum()[0] / comp._sim_counter

df = pd.DataFrame.from_dict(ff_cf).T

# Calculate the total row and append to the df
total = {
    "fm": "total",
    "ie": df["ie"].prod(),
    "is": comp._sim_counter - df[["ff", "cf"]].sum().sum(),
    "ff": df["ff"].sum(),
    "cf": df["cf"].sum(),
}
df = df.append(total, ignore_index=True).set_index("fm")

# Calculate the ratio of cf to ff
mask_failures = (df["cf"] != 0) & (df["cf"] != 0)
for var in ["ff", "cf"]:
    df[var + "%"] = (
        (
            df.loc[mask_failures, var]
            / (df.loc[mask_failures, "cf"] + df.loc[mask_failures, "ff"])
            * 100
        )
        .astype(float)
        .round(2)
    )
    df[var + "%"].fillna("", inplace=True)

df["cf:ff (%)"] = df["ff%"].astype(str) + ":" + df["cf%"].astype(str)

# Add the cohort data if it is supplied
if cohort is not None:
    sample_size = comp._sim_counter / cohort["assets"].sum()
    df[["ff_pop", "cf_pop"]] = df[["ff", "cf"]] / sample_size

    df.loc["total", "ff_pop"] = df["ff_pop"].sum()
    df.loc["total", "cf_pop"] = df["cf_pop"].sum()
    df.loc["total", "cf_pop_annual_avg"] = df["cf_pop_annual_avg"].sum()
    df.loc["total", "cf_pop_annual_avg"] = df["cf_pop_annual_avg"].sum()
    df.loc["total", "total"] = cohort["assets"].sum()

# Format for display
col_order = [
    "fm",
    "ie",
    "cf:ff (%)",
    "is",
    "ff",
    "cf",
    'ff_pop_annual_avg',
    'cf_pop_annual_avg',
    "ff_pop",
    "cf_pop",
    "total",
]
df = df.reset_index().reindex(columns=col_order)

df["ie"] = df["ie"].mul(100)
df.rename(columns={"ie": "ie (%)"}, inplace=True)

In [32]:
df

,fm,ie (%),cf:ff (%),is,ff,cf,ff_pop_annual_avg,cf_pop_annual_avg,ff_pop,cf_pop,total
0,early_life,18,:,97,3,0,434.16,0,35247.9,0,NaN
1,random,18,:,93,7,0,1041.37,0,82245,0,NaN
2,slow_aging,95.4,91.67:8.33,88,11,1,34.11,227.83,129242,11749.3,NaN
3,fast_aging,95.4,93.1:6.9,71,27,2,2263.15,116.99,317231,23498.6,NaN
4,total,2.94878,94.12:5.88,49,48,3,NaN,689.64,1.12793e+06,70495.7,1174929.0


assets    226315.0
dtype: float64